In [2]:
import openmm.app
from openff.toolkit import Molecule
from openff.units import Quantity, unit
from openmmforcefields.generators import GAFFTemplateGenerator
from rich.pretty import pprint

molecule = Molecule.from_smiles("O=S(=O)(N)c1c(Cl)cc2c(c1)S(=O)(=O)NCN2")
molecule.generate_conformers(n_conformers=1)

topology = molecule.to_topology()
topology.box_vectors = Quantity([4, 4, 4], unit.nanometer)

gaff = GAFFTemplateGenerator(molecules=molecule)

# If using this alongside another force field, we'd load that in here. But
# in this case, we're only using GAFF for this one molecule, so a "blank"
# force field is a sufficient starting point
forcefield_gaff = openmm.app.ForceField()
forcefield_gaff.registerTemplateGenerator(gaff.generator)

system_gaff = forcefield_gaff.createSystem(
    topology=topology.to_openmm(),
    nonbondedMethod=openmm.app.PME,
)


sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information availa

In [3]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [4]:
from openff.interchange import Interchange

imported = Interchange.from_openmm(
    topology=topology.to_openmm(),
    system=system_gaff,
    positions=molecule.conformers[0].to_openmm(),
)

In [5]:
from openff.interchange.drivers.openmm import (
    _get_openmm_energies,
    _process,
    get_openmm_energies,
)

energy_openmmforcefields = _process(
    _get_openmm_energies(
        system=system_gaff,
        box_vectors=topology.to_openmm().getPeriodicBoxVectors(),
        positions=molecule.conformers[0].to_openmm(),
        round_positions=None,
        platform="Reference",
    ),
    system=system_gaff,
    combine_nonbonded_forces=True,
    detailed=False,
)

pprint(energy_openmmforcefields)

EnergyReport(
│   energies={
│   │   'Bond': <Quantity(40.71813091571544, 'kilojoule / mole')>,
│   │   'Angle': <Quantity(80.74570857756217, 'kilojoule / mole')>,
│   │   'Torsion': <Quantity(225.26268867531294, 'kilojoule / mole')>,
│   │   'Nonbonded': <Quantity(-2033.6213556604541, 'kilojoule / mole')>
│   }
)

In [6]:
energy_imported = get_openmm_energies(imported, detailed=False)

pprint(energy_imported)

EnergyReport(
│   energies={
│   │   'Bond': <Quantity(40.718130915715236, 'kilojoule / mole')>,
│   │   'Angle': <Quantity(80.74570857756224, 'kilojoule / mole')>,
│   │   'Torsion': <Quantity(225.26268867531294, 'kilojoule / mole')>,
│   │   'Nonbonded': <Quantity(-2033.6237574776846, 'kilojoule / mole')>
│   }
)

In [7]:
energy_imported.compare(
    energy_openmmforcefields,
    {"Nonbonded": abs(energy_openmmforcefields["Nonbonded"] * 1e-5)},
)

energy_imported.diff(energy_openmmforcefields)

{'Bond': <Quantity(-2.0605739337042905e-13, 'kilojoule / mole')>,
 'Angle': <Quantity(7.105427357601002e-14, 'kilojoule / mole')>,
 'Torsion': <Quantity(0.0, 'kilojoule / mole')>,
 'Nonbonded': <Quantity(-0.0024018172305204644, 'kilojoule / mole')>}

In [9]:
from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange

paracetamol = Molecule.from_smiles("CC(=O)NC1=CC=C(C=C1)O")
paracetamol.generate_conformers(n_conformers=1)

topology = Topology.from_molecules([paracetamol])

force_field = ForceField("openff-2.0.0.offxml")

interchange = Interchange.from_smirnoff(force_field, topology)

openmm_system = interchange.to_openmm()
openmm_topology = interchange.to_openmm_topology()
openmm_positions = interchange.positions.to_openmm()

sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by /bin/bash)
sh: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information available (required by sh)
/bin/bash: /home/suncc/micromamba/envs/marimo/lib/libtinfo.so.6: no version information availa

In [11]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

pdb = PDBFile('input.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

1 warning generated.


#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-142556.096577,289.81659788471455
1000,-142556.096577,289.81659788471455
2000,-140692.4737742656,301.18886172319156
3000,-141264.2909129375,296.811340929822
4000,-141165.75429184374,302.30121211508214
5000,-140656.2069285625,303.02363780660755
6000,-140322.1639598125,303.5687245381905
7000,-140494.27772934374,303.6949771942317
8000,-140437.32704575,299.89088220730275
9000,-140722.8983348125,300.4815501653929
10000,-140868.52504379686,302.54416930173204


In [47]:
# Create an OpenFF Molecule object for benzene from SMILES
from openff.toolkit import Molecule

molecule = Molecule.from_file('/home/suncc/Code/pub/himatcal/examples/EleCal/openMM/EC.mol')


# Create the GAFF template generator
from openmmforcefields.generators import (
    GAFFTemplateGenerator,
)
gaff = GAFFTemplateGenerator(molecules=molecule)
gaff.add_molecules(molecule)


In [48]:

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
from openmm.app import ForceField

forcefield = ForceField(
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber/tip3p_HFE_multivalent.xml",
)
# Register the GAFF template generator
forcefield.registerTemplateGenerator(gaff.generator)

# You can now parameterize an OpenMM Topology object that contains the specified molecule.
# forcefield will load the appropriate GAFF parameters when needed, and antechamber
# will be used to generate small molecule parameters on the fly.
pdbfile = PDBFile("/home/suncc/Code/pub/himatcal/examples/EleCal/openMM/output.pdb")

# Create the system
system = forcefield.createSystem(topology)

Did not recognize residue EC; did you forget to call .add_molecules() to add it?


ValueError: No template found for residue 1 (EC).  The set of atoms is similar to ACE, but it is missing 3 atoms.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template